In [264]:
import torch
import torch.nn as nn
import numpy as np
import scipy.io 
import random
import math
import matplotlib.pyplot as plt
import torch.nn.functional as F
import os
import seaborn as sn
import pandas as pd
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
os.environ['KMP_DUPLICATE_LIB_OK']='True' 

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
import torch.optim as optim

Import Dataset

In [2]:
train_dl_origin = torch.load('Dataset/train_dl.pt')
valid_dl_origin = torch.load('Dataset/valid_dl.pt')

train_CSI = train_dl_origin.dataset[:][0]
train_label = train_dl_origin.dataset[:][1][:,0:2]
train_x_label = train_label[:,0]
train_y_label = train_label[:,1]

valid_CSI = valid_dl_origin.dataset[:][0]
valid_label = valid_dl_origin.dataset[:][1][:,0:2]
valid_x_label = valid_label[:,0]
valid_y_label = valid_label[:,1]

Seperate modulus and angle values, then combine into 1 dataset

In [3]:
train_a_raw = torch.angle(train_CSI).squeeze()
train_m_raw = torch.abs(train_CSI).squeeze()
valid_a_raw = torch.angle(valid_CSI).squeeze()
valid_m_raw = torch.abs(valid_CSI).squeeze()
train_am_raw = torch.stack((train_a_raw, train_m_raw), dim=3)
valid_am_raw = torch.stack((valid_a_raw, valid_m_raw), dim=3)

Reshape to matrix

In [4]:
train_am_raw = train_am_raw.reshape((train_am_raw.shape[0], 4*1632*2))
valid_am_raw = valid_am_raw.reshape((valid_am_raw.shape[0], 4*1632*2))

Normalize to mean 0 std 1

In [5]:
mean = torch.mean(train_am_raw, dim=0)
std = torch.std(train_am_raw, dim=0)
train_am = (train_am_raw-mean)/(std)
valid_am = (valid_am_raw-mean)/(std)

Linear Regression

In [276]:
from sklearn.linear_model import LinearRegression
lr_x = LinearRegression()
lr_x.fit(train_am, train_x_label)
lr_y = LinearRegression()
lr_y.fit(train_am, train_y_label)

LinearRegression()

In [277]:
pred_train_x = lr_x.predict(train_am)
pred_train_y = lr_y.predict(train_am)
print("M.A.P.E train x: ", mean_absolute_percentage_error(train_x_label, pred_train_x))
print("M.A.P.E train y: ", mean_absolute_percentage_error(train_y_label, pred_train_y))
print("M.S.E. train x: ", mean_squared_error(train_x_label, pred_train_x))
print("M.S.E. train y: ", mean_squared_error(train_y_label, pred_train_y))

M.A.P.E train x:  0.28542122
M.A.P.E train y:  0.2976161
M.S.E. train x:  2.6430402
M.S.E. train y:  1.0316823


In [278]:
pred_valid_x = lr_x.predict(valid_am)
pred_valid_y = lr_y.predict(valid_am)
print("M.A.P.E valid x: ", mean_absolute_percentage_error(valid_x_label, pred_valid_x))
print("M.A.P.E valid y: ", mean_absolute_percentage_error(valid_y_label, pred_valid_y))
print("M.S.E. valid x: ", mean_squared_error(valid_x_label, pred_valid_x))
print("M.S.E. valid y: ", mean_squared_error(valid_y_label, pred_valid_y))

M.A.P.E valid x:  2.556207
M.A.P.E valid y:  3.145411
M.S.E. valid x:  167.06215
M.S.E. valid y:  68.51464


SVM

In [279]:
from sklearn.svm import SVR
svr_x = SVR()
svr_x.fit(train_am, train_x_label)
svr_y = SVR()
svr_y.fit(train_am, train_y_label)

SVR()

In [280]:
pred_train_x = svr_x.predict(train_am)
pred_train_y = svr_y.predict(train_am)
print("M.A.P.E train x: ", mean_absolute_percentage_error(train_x_label, pred_train_x))
print("M.A.P.E train y: ", mean_absolute_percentage_error(train_y_label, pred_train_y))
print("M.S.E. train x: ", mean_squared_error(train_x_label, pred_train_x))
print("M.S.E. train y: ", mean_squared_error(train_y_label, pred_train_y))

KeyboardInterrupt: 

In [ ]:
pred_valid_x = svr_x.predict(valid_am)
pred_valid_y = svr_y.predict(valid_am)
print("M.A.P.E valid x: ", mean_absolute_percentage_error(valid_x_label, pred_valid_x))
print("M.A.P.E valid y: ", mean_absolute_percentage_error(valid_y_label, pred_valid_y))
print("M.S.E. valid x: ", mean_squared_error(valid_x_label, pred_valid_x))
print("M.S.E. valid y: ", mean_squared_error(valid_y_label, pred_valid_y))

Decision Tree

In [217]:
from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor(max_depth=10)  # when max_depth is not set, it overfits, with depth=29
dt.fit(train_am, train_x_label)

DecisionTreeRegressor(max_depth=10)

In [231]:
max_depths = [10, 15, 20, 25, 30]
min_splits = [2, 4, 6, 8, 10]
results = [] #(tree itself, (depth, split), (train % err, valid % err))
for max_depth in max_depths:
    for min_split in min_splits:
        dt = DecisionTreeRegressor(max_depth=max_depth, min_samples_split=min_split)
        dt.fit(train_am, train_x_label)
        pred_train_x = dt.predict(train_am)
        train_err_x = mean_absolute_percentage_error(train_x_label, pred_train_x)
        pred_valid_x = dt.predict(valid_am)
        valid_err_x = mean_absolute_percentage_error(valid_x_label, pred_valid_x)
        results.append((dt, (max_depth, min_splits), (train_err_x, valid_err_x)))

In [269]:
min = 1000
min_train = 1000
dt_x = None
for result in results:
    if result[2][1] < min:
        min = result[2][1]
        min_train = result[2][0]
        dt_x = result[0]
min_train, min

(0.006232623054380457, 0.4579651457707525)

In [271]:
print("Min_samples_split:",dt_x.min_samples_split,"\nMax depth", dt_x.max_depth)

Min_samples_split: 4 
Max depth 25


Train it on the y values

In [273]:
dt_y = DecisionTreeRegressor(max_depth = dt_x.max_depth, min_samples_split=dt_x.min_samples_split)
dt_y.fit(train_am, train_y_label)

DecisionTreeRegressor(max_depth=25, min_samples_split=4)

In [274]:
pred_train_x = dt_x.predict(train_am)
pred_train_y = dt_y.predict(train_am)
print("M.A.P.E train x: ", mean_absolute_percentage_error(train_x_label, pred_train_x))
print("M.A.P.E train y: ", mean_absolute_percentage_error(train_y_label, pred_train_y))
print("M.S.E. train x: ", mean_squared_error(train_x_label, pred_train_x))
print("M.S.E. train y: ", mean_squared_error(train_y_label, pred_train_y))

M.A.P.E train x:  0.006232623054380457
M.A.P.E train y:  0.006992053435375219
M.S.E. train x:  0.08776657553184125
M.S.E. train y:  0.03687887105764928


In [275]:
pred_valid_x = dt_x.predict(valid_am)
pred_valid_y = dt_y.predict(valid_am)
print("M.A.P.E valid x: ", mean_absolute_percentage_error(valid_x_label, pred_valid_x))
print("M.A.P.E valid y: ", mean_absolute_percentage_error(valid_y_label, pred_valid_y))
print("M.S.E. valid x: ", mean_squared_error(valid_x_label, pred_valid_x))
print("M.S.E. valid y: ", mean_squared_error(valid_y_label, pred_valid_y))

M.A.P.E valid x:  0.4579651457707525
M.A.P.E valid y:  0.45093538697782604
M.S.E. valid x:  22.204276844652814
M.S.E. valid y:  9.963308777646168


Neural Network

In [181]:
import torch
import torch.nn as nn

class Model(nn.Module):
    def __init__(self, input_size):
        super(Model, self).__init__()
        
        self.fcs = nn.ModuleList([nn.Linear(input_size, 3000), 
                                  nn.Linear(3000,1000), 
                                  nn.Linear(1000, 100),
                                  nn.Linear(100, 20)])
        self.output = nn.Linear(20, 2)
        self.dropout = nn.Dropout(.1)
        self.elu = nn.ELU()  

    def forward(self, x):
        # Forward pass through the network
        for fc in self.fcs:
            x = self.elu(self.dropout(fc(x)))
        x = self.output(x)
        x = self.dropout(x)
        return x
        
        
input_size = train_am.shape[1]
model = Model(input_size)

print(model)

Model(
  (fcs): ModuleList(
    (0): Linear(in_features=13056, out_features=3000, bias=True)
    (1): Linear(in_features=3000, out_features=1000, bias=True)
    (2): Linear(in_features=1000, out_features=100, bias=True)
    (3): Linear(in_features=100, out_features=20, bias=True)
  )
  (output): Linear(in_features=20, out_features=2, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (relu): ELU(alpha=1.0)
)


In [183]:
model = Model(input_size)
dataset = TensorDataset(train_am, train_label.to(torch.float32))
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)
val_dataset = TensorDataset(valid_am, valid_label.to(torch.float32))
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True) 

loss_fn = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)
#optimizer = optim.Adam(model.parameters(), lr=0.01)
#tried adam, but was slower and did much worse

best_loss = 5 # best validation loss achieved in all models
num_epochs = 200
for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    count = 0
    for inputs, targets in dataloader:
        count += 1
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        running_loss += loss
        
        loss.backward()
        optimizer.step()
    train_loss = running_loss/count
    
    model.eval()
    running_loss = 0
    count = 0
    for inputs, targets in val_dataloader:
        count += 1
        with torch.no_grad():
            running_loss += loss_fn(model(inputs), targets)
    val_loss = running_loss / count
    if val_loss < best_loss:
        best_loss = val_loss
        #torch.save(model.state_dict(), 'best-model-parameters-coordinates.pt')    
    
    print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {train_loss}, Test Loss: {val_loss}')

print('Training complete.')

Epoch [1/200], Training Loss: 133.4093780517578, Test Loss: 60.88287353515625
Epoch [2/200], Training Loss: 90.5941162109375, Test Loss: 54.6991081237793
Epoch [3/200], Training Loss: 76.77713775634766, Test Loss: 40.30664825439453
Epoch [4/200], Training Loss: 72.37869262695312, Test Loss: 34.32225799560547
Epoch [5/200], Training Loss: 66.85469055175781, Test Loss: 36.408424377441406
Epoch [6/200], Training Loss: 62.7216796875, Test Loss: 42.368221282958984
Epoch [7/200], Training Loss: 57.934818267822266, Test Loss: 21.906322479248047
Epoch [8/200], Training Loss: 56.797508239746094, Test Loss: 24.221954345703125
Epoch [9/200], Training Loss: 52.85347366333008, Test Loss: 20.355812072753906
Epoch [10/200], Training Loss: 52.529762268066406, Test Loss: 18.508981704711914
Epoch [11/200], Training Loss: 51.120758056640625, Test Loss: 13.629170417785645
Epoch [12/200], Training Loss: 47.83685302734375, Test Loss: 13.20617389678955
Epoch [13/200], Training Loss: 49.1103515625, Test Loss:

Using mean absolute percentage error and MSE to see how well the model did

Decent error on training (20%), Best I have gotten so far on validation (50%)

Good MSE on validation, around 5

In [187]:
model.eval()
with torch.no_grad():
    train_out = model(train_am).detach().numpy()
    val_out = model(valid_am).detach().numpy()
mean_absolute_percentage_error(train_label, train_out), mean_absolute_percentage_error(valid_label, val_out)

(0.21980761, 0.49818236)

In [195]:
loss_fn(torch.tensor(train_out, dtype=torch.float32), train_label).item(),loss_fn(torch.tensor(val_out, dtype=torch.float32), valid_label).item()

(1.9301880598068237, 5.239727973937988)